## Astrometry catalog (DEVELOPMENT, DOESN'T RUN....)

Process an association and derive an astrometric catalog to send to the `astrometry_db` database.

For fields that don't have a good existing astrometric reference (i.e., the pointings *not* in COSMOS, GOODS), derive a catalog from one JWST filter, e.g., F444W, and then align all other JWST filters to that one.

In [2]:
import os
import glob

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

from grizli import utils, prep
from grizli.aws import db, visit_processor

In [8]:
# Find PANORAMIC visits that haven't been processed

panoramic_assoc = db.SQL("""select assoc_name, max(status) as status, max(filter) as filter
from assoc_table
where status != 2 and filter in ('F444W-CLEAR')
and proposal_id in ('2514')
GROUP BY assoc_name
"""
) 

for assoc in panoramic_assoc['assoc_name']:
    print(assoc)
    
    if False:
        visit_processor.process_visit(assoc, sync=False, clean=False)


j100736p2109_indef-f444w_00023
j100736p2109_indef-f444w_00026


In [ ]:
# Send visit
mag_lim = (17, 26)
flux_radius = (1,10)
sn = 20

max_count = 1000

HOME = os.getcwd()

assoc_list = [assoc in panoramic_assoc['assoc_name']]

for assoc in assoc_list:
    os.chdir(HOME)
    
    if os.path.exists(f'{assoc}/Prep'):
        os.chdir(f'{assoc}/Prep')
        files = glob.glob('*cat.fits')
        for file in files:
            tab = utils.read_catalog(file)
            for c in list(tab.colnames):
                tab.rename_column(c, c.lower())

            tab['mag'] = tab['mag_auto']
            if True:
                #clip = tab['mag'] < mag_lim[1]
                clip = tab['mag'] > mag_lim[0]
                clip &= tab['mask_aper_1'] == 0
                clip &= tab['flux_radius'] > flux_radius[0]
                clip &= tab['flux_radius'] < flux_radius[1]
                clip &= tab['flux_aper_1'] / tab['fluxerr_aper_1'] > sn
                
                tab = tab[clip]
                so = np.argsort(tab['mag'])
                tab = tab[so]
                
                if max_count > 0:
                    tab = tab[:max_count]
                    
            plt.scatter(tab['mag_auto'], tab['flux_radius'], alpha=0.2)
            plt.ylim(0, 10)
            plt.xlim(12,28)
            
            if len(files) == 1:
                src = assoc+'_cat'
            else:
                src = os.path.basename(file).split('.cat')[0]
            
            src = src.replace('indef-','')
            
            tab['src'] = src
            try:
                ok = np.isfinite(tab['mag'].filled(np.nan))
                tab = tab[ok]
            except:
                pass

            print(f'Add {len(tab)} objects to astrometry_referenc for {assoc}/{file} src={src}')
            
            prep.table_to_regions(tab, f'/tmp/{src}.reg')
            
            # Set this to True to actually send the table!
            if False:
                print(f'clean {src}')
                db.execute(f"delete from astrometry_reference where src='{src}'")
                print(f'upload {src}')
                db.send_to_database('astrometry_reference', tab['ra','dec','src','mag'], if_exists='append')


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a56be2b0-05e9-4804-afc1-e2fa4997b5d9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>